In [2]:
import signal
import pandas as pd
import math

In [14]:
# todo ATTENTION!!!!!: got help from internet using shahdeven git

import math


class CSP:
    def __init__(self, number_of_marks):
        """
        Here we initialize all the required variables for the CSP computation,
        according to the number of marks.
        """
        # Your code here
        self.number_of_marks = number_of_marks
        self.current_length = 0  # Update this line
        self.variables = []  # Update this line
        self.differences = [[]]  # Update this line

    # def assign_value(self, i, v):
    #    """
    #    assign a value v to variable with index i
    #    """
    #    # Your code here
    #    pass

    def check_constraints(self, allocateFlag, distancesNew, distancesTemp, mCurrent, mPos, positions):
        for i in range(1, mCurrent):
            distance = abs(mPos - positions[i])

            if distance in distancesNew:
                allocateFlag = False
                break
            distancesNew.append(distance)
            distancesTemp.append(distance)
        return allocateFlag

    def backtrack(self, mark_positions, mark_domains, current_mark, distancesCurrent, length):

        if -1 not in mark_positions.values():
            return [mark_positions[current_mark - 1], mark_positions]

        for i in mark_domains.keys():
            if len(mark_domains[i]) == 0:
                return [-1, mark_positions]

        for position in mark_domains[current_mark]:
            allocateFlag = True

            distancesNew = list(distancesCurrent)
            distancesTemp = []

            allocateFlag = self.check_constraints(allocateFlag, distancesNew, distancesTemp, current_mark, position,
                                                  mark_positions)
            if not allocateFlag:
                continue

            positionsNew = dict(mark_positions)
            positionsNew[current_mark] = position

            domainsNew = {}
            for elem in mark_domains.keys():
                domainsNew[elem] = list(mark_domains[elem])

            self.forward_check(length, distancesTemp, domainsNew, current_mark, mark_positions, positionsNew,
                               mark_positions, position)

            end_flag = False
            for i in domainsNew.keys():
                if len(domainsNew[i]) == 0:
                    end_flag = True
            if not end_flag:
                return_len, retPositions = self.backtrack(positionsNew, domainsNew, current_mark + 1, distancesNew,
                                                          length)
                if return_len != -1:
                    return return_len, retPositions
        return -1, mark_positions



    def forward_check(self, length, distancesTemp, domainsNew, current_mark, positions, positionsNew,
                      mark_positions, position):

        for i in range(current_mark + 1, len(mark_positions.keys()) + 1):
            if position in domainsNew[i]:
                domainsNew[i].remove(position)

        for item in positionsNew.values():
            if item == -1:
                break
            for elem in distancesTemp:
                position_to_remove1 = item + elem
                position_to_remove2 = item - elem
                if position_to_remove1 > length and position_to_remove2 < 0:
                    continue
                for i in range(current_mark + 1, len(positions.keys()) + 1):
                    if  position_to_remove1 <= length and position_to_remove1 in domainsNew[i]:
                        domainsNew[i].remove(position_to_remove1)
                    if  position_to_remove2 <= length and position_to_remove2 in domainsNew[i]:
                        domainsNew[i].remove(position_to_remove2)
                #if position_to_remove <= length:
                #    for i in range(current_mark + 1, len(positions.keys()) + 1):
                #        if position_to_remove in domainsNew[i]:
                #            domainsNew[i].remove(position_to_remove)
#
                #position_to_remove = item - elem
                #if position_to_remove >= 0:
                #    for i in range(current_mark + 1, len(positions.keys()) + 1):
                #        if position_to_remove in domainsNew[i]:
                #            domainsNew[i].remove(position_to_remove)

    def find_minimum_length(self) -> int:
        """
        This is the main function of the class.
        First, we start by assigning an upper bound value to variable current_length.
        Then, using backtrack and forward_check functions, we decrease this value until we find
        the minimum required length.
        """

        mark_positions = {}

        for i in range(0, self.number_of_marks):
            mark_positions[i + 1] = -1

        self.current_length = math.ceil(self.number_of_marks * (self.number_of_marks - 1) / 2)
        if(self.number_of_marks>6):
            self.current_length  += (math.ceil((self.number_of_marks-1) / 2) + (self.number_of_marks-6))
        print(self.current_length)
        initialDomains = []
        for i in range(0, self.current_length + 1):
            initialDomains.append(i)
        mark_domains = {}

        while True:

            for i in range(0, self.number_of_marks):
                mark_domains[i + 1] = initialDomains.copy()
            lenght, mark_positions = self.backtrack(mark_positions, mark_domains, 1, [], self.current_length)
            if lenght != -1:
                self.variables = list(mark_positions.values())
                return lenght

            self.current_length += 1
            initialDomains.append(self.current_length)

            pass
        # return 0

    def get_variables(self) -> list:
        """
        Get variables array.
        """
        # No need to change
        return self.variables


In [15]:

def handler(signum, frame):
    raise Exception("End Of Running Time")


def run_tests():
    df = pd.read_csv("data.csv")
    df = df.reset_index()
    for index, row in df.iterrows():
        signal.signal(signal.SIGALRM, handler)
        signal.alarm(10)
        csp = CSP(int(row['marks']))
        print(f"Running Test {index}..., ", end=" ")
        try:
          result = csp.find_minimum_length()
          variables = csp.get_variables()
          #print(result)
          #print(variables)
          expected = int(row['len'])
          check_variables = check_output(variables, row['array'])
          accepted = result == expected and check_variables
        except:
            print("A")
            accepted = False
        if accepted:
            print(f"passed!")
        else:
            print(f"\u001B[31mfailed\u001B[0m")


def check_output(output, expected):

    expected = sorted(map(int, expected.split(" ")))
    output = sorted(output)
    if len(expected) != len(output):
        print("len")
        return False
    for i in range(len(expected)):
        if expected[i] != output[i]:
            print("val")
            return False
    return True


run_tests()


Running Test 0...,  0
passed!
Running Test 1...,  1
passed!
Running Test 2...,  3
passed!
Running Test 3...,  6
passed!
Running Test 4...,  10
passed!
Running Test 5...,  15
passed!
Running Test 6...,  25
passed!
Running Test 7...,  34
passed!
Running Test 8...,  43
A
failed
Running Test 9...,  54
A
failed
Running Test 10...,  65
A
failed
Running Test 11...,  78
A
failed
Running Test 12...,  91
A
failed
Running Test 13...,  106
A
failed
Running Test 14...,  121
A
failed
Running Test 15...,  138
A
failed
Running Test 16...,  155
A
failed
Running Test 17...,  174
A
failed
Running Test 18...,  193
A
failed
Running Test 19...,  214
A
failed
Running Test 20...,  235
A
failed
Running Test 21...,  258
A
failed
Running Test 22...,  281
A
failed
Running Test 23...,  306
A
failed
Running Test 24...,  331
A
failed
Running Test 25...,  358
A
failed
